In [3]:
import pandas as pd
from lightfm import LightFM
import lightfm.cross_validation
from scipy.sparse import coo_matrix
from lightfm.evaluation import auc_score
import plotly.express as px
import re
import json
from tqdm.auto import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
import numpy as np
import skopt
from lightfm.cross_validation import random_train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [7]:
courses_features = pd.read_csv('courses_features.csv',sep = '^',index_col=0)
courses_features_1 = pd.read_csv('courses_features_1.csv',sep = '^',index_col=0)
interactions_matrix = pd.read_csv('interactions_matrix.csv',index_col=0)
df_interactions = pd.read_csv('data_t_row_6m.csv')

Оставим в матрице взаимодействий только индексы курсов, так как все ссылки идентичны

In [4]:
interactions_matrix.columns = list(map(lambda x: int(re.findall('[0-9]+',x)[0]),interactions_matrix.columns))

In [51]:
interactions_matrix = interactions_matrix.reindex(sorted(interactions_matrix.columns), axis=1)
courses_features = courses_features.reindex(sorted(courses_features.index),axis = 0)

Посмотрим сколько курсов таких, что о них есть информация о взаимодействии, но нет их признаков

In [52]:
with open('./hse_dpo_portal_data.json') as json_file:
    temp = pd.DataFrame(json.load(json_file)['data'])
sum(list(map(lambda x: x not in temp.id.values ,list(interactions_matrix.index))))

Удалим из взаимодействий те курсы, о которых у нас нет информации, чтобы не было добавлены много курсов с полностью идентичными нулевыми признаками.

In [5]:
cols_to_delete = list(filter(lambda x: x not in list(courses_features.id.values),interactions_matrix.columns))
interactions_matrix = interactions_matrix.drop(cols_to_delete,axis = 1)

Так как мы удалили некоторые курсы, то у нас могли получиться пользователи, которые не взаимодействовали ни с какими курсами, удалим их

In [6]:
interactions_matrix = interactions_matrix.loc[(interactions_matrix!=0).any(axis=1)]

Отсортируем курсы в таблице признаков, так чтобы они согласовались с порядком курсов в таблице взаимодействий

In [7]:
courses_features = courses_features.loc[courses_features['id'].isin(list(interactions_matrix.index))]
courses_features.id= courses_features.id.astype("category")
courses_features.id = courses_features.id.cat.set_categories(np.unique(list(interactions_matrix.columns) +
                                     list(filter(lambda x: x not in interactions_matrix.columns, courses_features.id.values))))
courses_features = courses_features.sort_values(by=['id'])

Установим id курса как индекс

In [8]:
courses_features = courses_features.set_index('id')

Нормализуем признаки курсов для того чтобы получить более устойчивую модель и лучшие результаты

In [9]:
scaler = StandardScaler()
scaler.fit(courses_features)
courses_features = pd.DataFrame(scaler.transform(courses_features),index = courses_features.index, columns=courses_features.columns)

Приведем наши матрицы к нужному виду, а также выделим подматрицу из нетекстовых признаков курсов, будем ее использовать для подбора гиперпараметров для более быстрого обучения и подбора

In [3]:
temp_features_coo = coo_matrix(courses_features[courses_features.columns[:63]])
interactions_matrix_coo = coo_matrix(interactions_matrix)
interactions_matrix_without_weights_coo = coo_matrix(interactions_matrix.transform(lambda x: (x>0).astype(float)))
courses_features_coo = coo_matrix(courses_features)

Разделим данные на обучающую и тестовую выборки по дате в отношении 4:1

In [4]:
train_intersections, test_intersections = lightfm.cross_validation.random_train_test_split(interactions_matrix_coo,
                                                                            test_percentage = 0.2, random_state = 122333)
train_intersections_without_weights, test_intersections_without_weights = lightfm.cross_validation.random_train_test_split(
                                                                            interactions_matrix_without_weights_coo,
                                                                            test_percentage = 0.2, random_state = 122333)

In [6]:
pd.DataFrame.sparse.from_spmatrix(train_intersections).to_csv('train_intersections.csv')
pd.DataFrame.sparse.from_spmatrix(test_intersections).to_csv('test_intersections.csv')

Попробуем обучить модель не подбирая лучшие гиперпараметры, будем использовать матрицу взаимодействия с весами и полный набор признаков

In [ ]:
model = lightfm.LightFM(random_state = 122333)
model_with_weights_full_features = model.fit(train_intersections_without_weights,sample_weight = train_intersections,item_features = courses_features_coo, verbose = True,epochs = 30)

Проверим auc_score получившейся модели

In [200]:
auc_score = lightfm.evaluation.auc_score(model_with_weights_full_features,test_intersections,
                                         train_intersections_without_weights,item_features=courses_features_coo)
print('Mean auc_score = ', np.mean(auc_score))
fig = px.histogram(auc_score)
fig.update_layout(showlegend = False,title = 'Распределение auc_score модели с весами и полными признаками без гиперпараметров у разных пользователей')
fig.show()

Mean auc_score =  0.75172484


Загрузим датасет с помеченными классами для пользователей, выставленными на основе их активности на платформе и посмотрим на то как получившаяся модель работает для тех или иных классов

In [204]:
users_type = pd.read_csv('./user_to_split_type.csv',index_col=0)
users_type = users_type.loc[list(interactions_matrix.index)]
temp_test_intersections = pd.DataFrame.sparse.from_spmatrix(test_intersections)

users_type_score = users_type.copy().reset_index().join(pd.DataFrame(auc_score,index = 
                                    temp_test_intersections.loc[temp_test_intersections.sum(axis = 1) != 0].index))
users_type_score.columns = ['id','class','auc_score']
users_type_score = users_type_score.set_index('id')
users_type_score

,class,auc_score
id,,
166826354536403284,0,NaN
1659090028176813185,0,0.705882
1662653630805002670,0,NaN
1667388264394057150,4,NaN
1666014310920836055,0,NaN
...,...,...
1662407034258674228,4,NaN
1665826640406257416,0,NaN
166680056086770561,0,NaN


Разобьем auc_score на интервалы для лучшей визуализации результатов

In [205]:
users_type_score['auc_score_bins'] = pd.cut(users_type_score.auc_score,15)

Построим график распределения auc_score по пользователям с разбивкой на классы

In [206]:
def build_histogram_with_classes():
    temp = users_type_score.reset_index().groupby(['class','auc_score_bins'])['id'].agg('count').reset_index()
    temp['auc_score_bins'] = temp['auc_score_bins'].astype(str)
    temp['class'] = temp['class'].astype(str)
    temp = temp.rename({'id':'count'},axis = 'columns')
    fig = px.bar(temp,
                x = 'auc_score_bins',
                y = 'count',
                template = 'plotly_dark',
                color = 'class',
                title = 'Auc_score с разбивкой на классы пользователей', 
                color_discrete_sequence=['#0d0887','#46039f','#7201a8','#9c179e','#bd3786','#d8576b','#ed7953','#fb9f3a','#fdca26','#f0c821','#faf921']
                )
    return fig

In [207]:
build_histogram_with_classes().show()

Распределение получилось достаточно равномерным, в том смысле, что в каждом столбце присутствуют пользователи из разных классов в примерно равных пропорциях, то есть мы одинаково хорошо составляем рейтинги для каждого класса 

Теперь обучим модель с весами в матрице взаимодействий и будем использовать только нетекстовые признаки

In [208]:
model_with_weights_short_features = model.fit(train_intersections_without_weights,sample_weight = train_intersections,item_features = temp_features_coo, verbose = True,epochs = 30)

Epoch: 100%|██████████| 30/30 [00:22<00:00,  1.33it/s]


In [211]:
auc_score = lightfm.evaluation.auc_score(model_with_weights_short_features,test_intersections_without_weights,
                                         train_intersections_without_weights,item_features=temp_features_coo)
print('Mean auc_score = ', np.mean(auc_score))
fig = px.histogram(auc_score)
fig.update_layout(showlegend = False,title = 'Распределение auc_score модели с весами без текстовых признаков без гиперпараметров у разных пользователей')
fig.show()

Mean auc_score =  0.7157294


In [212]:
users_type_score['auc_score'] = pd.DataFrame(index=temp_test_intersections.index).join(pd.DataFrame(auc_score,index = temp_test_intersections.loc[temp_test_intersections.sum(axis = 1) != 0].index)).values
users_type_score['auc_score_bins'] = pd.cut(users_type_score.auc_score,15)
build_histogram_with_classes().show()

У этой модели результаты немного хуже, средний auc_score ниже, чем у предыдущей модели, распределение не так сильно смещено вправо, больше пользователей из классов активных пользователей имеют низкую метрику, большую часть высоких значений метрики занимают пользователи нулевого класса (имеют маленькую заполненность, максимум, среднее и сумму)

Теперь обучим модель без весов взаимодействий, с полным набором признаков

In [213]:
model_without_weights_full_features = model.fit(train_intersections_without_weights,sample_weight=train_intersections, item_features = courses_features_coo, verbose = True,epochs = 30)

Epoch: 100%|██████████| 30/30 [03:21<00:00,  6.71s/it]


In [214]:
auc_score = lightfm.evaluation.auc_score(model_without_weights_full_features,test_intersections_without_weights,
                                         train_intersections_without_weights,item_features=courses_features_coo)
print('Mean auc_score = ', np.mean(auc_score))
fig = px.histogram(auc_score)
fig.update_layout(showlegend = False,title = 'Распределение auc_score модели без весов с полным набором признаков без гиперпараметров у разных пользователей')
fig.show()

Mean auc_score =  0.7507116


In [215]:
users_type_score['auc_score'] = pd.DataFrame(index=temp_test_intersections.index).join(pd.DataFrame(auc_score,index = temp_test_intersections.loc[temp_test_intersections.sum(axis = 1) != 0].index)).values
users_type_score['auc_score_bins'] = pd.cut(users_type_score.auc_score,15)
build_histogram_with_classes().show()

Результат без весов получился немного лучше, скорее всего текущие веса добавляют много шума, следует подобрать их как гиперпараметры, чтобы они наилучшим образом соответствовали уровню реального взаимодействия пользователя с курсом

Подберем гиперпараметры для модели с текущими весами, чтобы можно было прогнать много комбинаций не будем использовать текстовые признаки, а также выбросим половину пользователей так, чтобы их доля классов в получившейся выборке осталась такой же (за исключением пользователей с классом 0, их удалим больше, так как они несут меньше информации и их подавляющее большинство)

In [216]:
short_interactions_matrix = interactions_matrix.copy()
short_interactions_matrix['class'] = users_type_score['class']
short_interactions_matrix,_ = train_test_split(short_interactions_matrix,train_size = 0.6, random_state=122333,stratify=short_interactions_matrix['class'])

In [217]:
short_interactions_matrix = short_interactions_matrix.drop(
                                train_test_split(short_interactions_matrix.loc[short_interactions_matrix['class'] == 0],
                                train_size = 0.8,random_state=122333)[1].index).drop(['class'],axis = 1)


In [218]:
short_interactions_matrix_coo = coo_matrix(short_interactions_matrix)
short_interactions_matrix_without_weights_coo = coo_matrix(short_interactions_matrix.transform(lambda x: (x>0).astype(float)))
short_train_interactions, short_test_interactions = random_train_test_split(short_interactions_matrix_coo,test_percentage=0.2, 
                                                                            random_state=122333)
short_train_interactions_without_weights, short_test_interactions_without_weights = random_train_test_split(short_interactions_matrix_without_weights_coo,test_percentage=0.2, 
                                                                            random_state=122333)

In [219]:
def get_params(size):
    for i in range(size):
        params =  {
                "no_components": (np.random.randint(6,30)),
                "learning_schedule": (np.random.choice(["adagrad", "adadelta"])),
                "loss": (np.random.choice(["bpr", "warp","logistic"])),
                "learning_rate": (np.random.exponential(0.1)),
                "max_sampled": (np.random.randint(4, 20)),
                "epochs" : (np.random.randint(30,100))
            }
        yield params

In [227]:
best_auc_score = 0
best_params_info = []
best_params = dict()
best_epochs = 0

In [243]:
def cross_validation(model,interactions_without_weights,interactions,items_features, cv, epochs):
    cv_info = []
    for i in range(cv):
        cv_dict = dict()
        train_interactions, test_interactions= random_train_test_split(interactions ,random_state = i)
        train_interactions_without_weights, _ = random_train_test_split(interactions_without_weights, random_state=i)
        model.fit(train_interactions_without_weights,item_features = items_features, sample_weight = train_interactions, epochs = epochs)
        auc = lightfm.evaluation.auc_score(model, test_interactions, train_interactions_without_weights, item_features = items_features)
        cv_dict['mean_score'] = auc.mean()
        cv_dict['median_score'] = np.median(auc)
        cv_dict['std'] = np.std(auc)
        cv_info.append(cv_dict)
    return cv_info, np.mean([score['mean_score'] for score in cv_info])

def tuning_hyperparams (train_interactions_without_weights,train_interactions,items_features, cv = 3, number_of_combinations = 12345):
    global best_auc_score, best_params_info, best_params, best_epochs
    np.random.seed(122333)
    for params in tqdm(get_params(number_of_combinations)):
        epochs = params.pop('epochs')
        model = LightFM(**params, random_state = 122333)
        curr_info, curr_auc_score = cross_validation(model,train_interactions_without_weights,train_interactions,items_features, cv, epochs)
        if  curr_auc_score > best_auc_score:
            best_auc_score = curr_auc_score
            best_params_info = curr_info
            best_params = params
            best_epochs = epochs
            with open('./best_parameters.json', 'w') as f:
                f.write(str(dict(auc_score = best_auc_score,epochs = best_epochs)))
                f.write(str(best_params))
    best_model = LightFM(**best_params, random_state = 122333)
    best_model.fit(train_interactions_without_weights,item_features = items_features, sample_weight = train_interactions, epochs = best_epochs)
    return best_model, best_params, best_auc_score, best_params_info

In [245]:
pd.DataFrame.sparse.from_spmatrix(short_train_interactions_without_weights).to_csv('./train_model/short_train_interactions_without_weights.csv', index=False)
pd.DataFrame.sparse.from_spmatrix(short_train_interactions).to_csv('./train_model/short_train_interactions.csv', index=False)
pd.DataFrame.sparse.from_spmatrix(temp_features_coo).to_csv('./train_model/temp_features_coo.csv', index=False)

In [ ]:
best_model, best_params, best_auc_score, best_params_info = tuning_hyperparams(short_train_interactions_without_weights, short_train_interactions,temp_features_coo)

In [ ]:
best_params = {'no_components': 29, 'learning_schedule': 'adagrad', 'loss': 'logistic', 'learning_rate': 0.0008240852584760048, 'max_sampled': 12}
best_model = LightFM(**best_params, random_state = 122333)
best_model.fit(train_intersections_without_weights,item_features = courses_features_coo, sample_weight = train_intersections, verbose=True, epochs = 300)
auc = lightfm.evaluation.auc_score(best_model, test_intersections, train_intersections_without_weights, item_features = courses_features_coo)
auc.mean()

# Collaborative Filtering

In [26]:
df_interactions = pd.read_csv('data_t_row_6m.csv')

In [3]:
interactions_matrix_without_weights = interactions_matrix.transform(lambda x: (x>0).astype(float))

In [1]:
import implicit

/home/cubik/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_intersections = pd.read_csv('train_intersections.csv',index_col=0)
test_intersections = pd.read_csv('test_intersections.csv',index_col=0)

In [9]:
train_intersections.columns = interactions_matrix.columns
train_intersections.index = interactions_matrix.index
test_intersections.columns = interactions_matrix.columns
test_intersections.index = interactions_matrix.index

In [10]:
test_intersections_without_weights = test_intersections.apply(lambda x: (x > 0).astype(float))
train_intersections_without_weights = train_intersections.apply(lambda x: (x > 0).astype(float))

In [11]:
train_intersections_without_weights_csr = coo_matrix(train_intersections_without_weights).tocsr()
test_intersections_without_weights_csr = coo_matrix(test_intersections_without_weights).tocsr()
train_intersections_csr = coo_matrix(train_intersections).tocsr()
test_intersections_csr = coo_matrix(test_intersections).tocsr()

In [12]:
from implicit import evaluation

In [13]:
interactions_matrix.columns = list(map(lambda x: int(re.findall('[0-9]+',x)[0]),interactions_matrix.columns))

Посмотрим на разные коллаборативные модели в данной библиотеке (без подбора гиперпараметров) и сравним их предсказания, и качество на тестовой выборке

1. ALS

In [14]:
model = implicit.als.AlternatingLeastSquares(factors=50,iterations=30)
model.fit(train_intersections_without_weights_csr)

100%|██████████| 30/30 [04:09<00:00,  8.31s/it]


In [15]:
def get_evaluations(model,K=10):
    p_at_k= implicit.evaluation.precision_at_k(model,train_intersections_without_weights_csr,test_intersections_without_weights_csr,K=K,show_progress = False)
    map_at_k = implicit.evaluation.mean_average_precision_at_k(model,train_intersections_without_weights_csr,test_intersections_without_weights_csr,show_progress = False,K=K)
    ndcg_at_k = implicit.evaluation.ndcg_at_k(model,train_intersections_without_weights_csr,test_intersections_without_weights_csr,show_progress = False,K=K)
    print(f'P@{K} = {p_at_k}')
    print(f'MAP@{K} = {map_at_k}')
    print(f'NDCG@{K} = {ndcg_at_k}')
    return p_at_k,map_at_k,ndcg_at_k

In [16]:
p_at_k_als,map_at_k_als, ndcg_at_k_als = get_evaluations(model) 

P@10 = 0.17285551191006018
MAP@10 = 0.0667724607413641
NDCG@10 = 0.09166413928379208


Для визуальной оценки рекомендаций возьмем за основу 12-го по количеству взаимодействий пользователя

In [17]:
user_index = interactions_matrix.apply(lambda x: (x>0).astype(int)).reset_index(drop = True).sum(axis = 1).sort_values(ascending = False).index[12]

Посмотрим на топ-15 курсов, с которыми он больше всего взаимодействовал в реальности

In [ ]:
list_interact = np.array(interactions_matrix.iloc[user_index].sort_values(ascending = False).head(15).index)
list_interact = list_interact[np.isin(list_interact,courses_features.id)]
courses_features.set_index('id').loc[list_interact].title.values

Посмотрим на топ-10 курсов, которые рекомендует модель с включением в список уже просмотренных

In [ ]:
def get_name_of_courses(list_recommended):
    list_recommended_1 = interactions_matrix[[interactions_matrix.columns[ind] for ind in list_recommended]].columns
    list_recommended_1 = list_recommended_1[np.isin(list_recommended_1,courses_features.id)]
    print(courses_features_1.set_index('id').loc[list_recommended_1].title.values)

In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr[user_index],filter_already_liked_items=False)[0])

['Коучинг: продвинутый курс' 'Теоретическая психология. Основы практики'
 'Управление проектами' 'Бизнес-медиация'
 'Деловые переговоры: правовые стратегии'
 'Экономика и управление предприятием'
 'Executive Master in Strategy «Стратегия бизнеса»'
 'Финансовый менеджмент'
 'Executive Master in Finance «Стратегическое управление финансами»']


Теперь посмотрим на рекомендации без просмотренных курсов

In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr[user_index],filter_already_liked_items=True)[0])

['IELTS Основной Курс / IELTS Regular Course'
 'Эффективные переговоры как инструмент коммуникационного процесса'
 'Анализ и оценка инвестиционных проектов'
 'Система управленческого учета и бюджетирования'
 'Современная финансовая аналитика компании: лучшие практики'
 'Проектное финансирование: передовой опыт и актуальные практики'
 'Финансовый менеджмент' 'Менеджер цифровых проектов / Digital manager'
 'Мастерство проектного управления'
 'Out of the box thinking в управлении проектами: ТРИЗ, дизайн-мышление и иные нестандартные проектные практики']


2. Bayesian Personalized Ranking

In [ ]:
model = implicit.cpu.bpr.BayesianPersonalizedRanking(factors=50,iterations=100)
model.fit(train_intersections_without_weights_csr)

100%|██████████| 100/100 [00:06<00:00, 16.48it/s, train_auc=99.18%, skipped=1.73%]


In [ ]:
p_at_k_bpr,map_at_k_bpr, ndcg_at_k_bpr = get_evaluations(model) 

P@10 = 0.18568470752143038
MAP@10 = 0.08589192875129725
NDCG@10 = 0.1082041883064482


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr[user_index],filter_already_liked_items=False)[0])

['Теоретическая психология. Основы практики' 'Управление тревогой'
 'Культура и нормы современного психологического тестирования'
 'Исследования и консультативная практика в психологии'
 'Креативное мышление для решения профессиональных задач'
 'Управление стрессом в современных условиях' 'Управление конфликтами'
 'Практические инструменты в психологическом консультировании']


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr[user_index],filter_already_liked_items=True)[0])

['Управление тревогой'
 'Исследования и консультативная практика в психологии'
 'Управление стрессом в современных условиях' 'Мастерство переговоров'
 'Санкционная политика в современном мире'
 'Бизнес-аналитика: тренинг по построению аналитических отчетов (Dashboards) в Power BI'
 'Управление изменениями в команде для руководителей']


3. Logistic Matrix Factorization

In [ ]:
model = implicit.cpu.lmf.LogisticMatrixFactorization(factors=50,iterations=200)
model.fit(train_intersections_without_weights_csr)

100%|██████████| 200/200 [01:08<00:00,  2.93it/s]


In [ ]:
p_at_k_log,map_at_k_log, ndcg_at_k_log = get_evaluations(model) 

P@10 = 0.09127498597109077
MAP@10 = 0.02539561143672168
NDCG@10 = 0.04158684274494699


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr[user_index],filter_already_liked_items=False)[0])

['Бухгалтер коммерческой организации' 'Коучинг: продвинутый курс'
 'Практический курс разговорного английского языка (уровень Intermediate)'
 'Налоговый консультант-советник'
 'Методика преподавания русского языка как иностранного'
 'Налоговый и финансовый консалтинг'
 'Операционные, инвестиционные решения и эффективность бизнеса'
 'Управленческий учет и бюджетирование как инструменты оперативного управления бизнесом'
 'Управление государственными и муниципальными закупками (108 часов, очно)']


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr,N = 10,filter_already_liked_items=True)[0])

Теперь пронаблюдаем рекомендации content-based моделей

1. Cosine Recommender (implicit)

In [ ]:
model = implicit.nearest_neighbours.CosineRecommender(K = 30)

In [ ]:
model.fit(train_intersections_without_weights_csr)

/home/cubik/.local/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.007283687591552734 seconds
  warnings.warn(
100%|██████████| 570/570 [00:00<00:00, 11474.37it/s]


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr,N = 10,filter_already_liked_items=True)[0])

['Стратегический маркетинг' 'Управление проектами'
 'Executive Master in Management «Руководитель предприятия»'
 'Экономика и управление предприятием'
 'Как настроить работу команды: инструменты руководителя'
 'Мастерство проектного управления'
 'Управление проектами в условиях неопределённости'
 'Системное стратегическое мышление: понять прошлое, описать будущее, определить  стратегию'
 'Адаптивные методы управления организацией в условиях неопределённости'
 'Продуктовый подход к развитию компании']


2. TF-IDF (implicit)

In [ ]:
model = implicit.nearest_neighbours.TFIDFRecommender(K = 30)
model.fit(train_intersections_without_weights_csr)

/home/cubik/.local/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.003163576126098633 seconds
  warnings.warn(
100%|██████████| 570/570 [00:00<00:00, 7133.11it/s]


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr,filter_already_liked_items=True)[0])

['Стратегический маркетинг' 'Управление проектами'
 'Executive Master in Management «Руководитель предприятия»'
 'Экономика и управление предприятием'
 'Как настроить работу команды: инструменты руководителя'
 'Мастерство проектного управления'
 'Управление проектами в условиях неопределённости'
 'Системное стратегическое мышление: понять прошлое, описать будущее, определить  стратегию'
 'Адаптивные методы управления организацией в условиях неопределённости']


3. BM25 (implicit)

In [ ]:
model = implicit.nearest_neighbours.BM25Recommender(K=30)
model.fit(train_intersections_without_weights_csr)

/home/cubik/.local/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0028192996978759766 seconds
  warnings.warn(
100%|██████████| 570/570 [00:00<00:00, 17290.34it/s]


In [ ]:
get_name_of_courses(model.recommend(user_index,train_intersections_without_weights_csr,filter_already_liked_items=True)[0])

['Стратегический маркетинг'
 'Executive Master in Management «Руководитель предприятия»'
 'Как настроить работу команды: инструменты руководителя'
 'Экономика и управление предприятием' 'Управление проектами'
 'Мастерство проектного управления'
 'Управление проектами в условиях неопределённости'
 'Системное стратегическое мышление: понять прошлое, описать будущее, определить  стратегию'
 'Адаптивные методы управления организацией в условиях неопределённости']
